# 전이학습(Transfer Learning)을 이용한 데이터 분류 프로젝트!
- 사전에 학습된(Pretrained) ResNet50모델 사용!
- 나만의 데이터셋(Custom dataset) 적용해보기
- 완전 연결 계층(fully connected layer) 수정
- 코랩을 이용하여 적용!
- 라즈베리파이로 이용해보기!

## 1. Settings
### 1) Import required libraries

## 코랩에 데이터를 올려보기!
### 구글 드라이브도 올려보세요!

In [ ]:
# from google.colab import files
# from google.colab import drive
# from google.colab.output import eval_js
# from imutils import paths
# from IPython.display import display, Javascript
# from base64 import b64decode

# uploaded = files.upload()
# drive.mount('/content/gdrive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from PIL import Image
from torch.optim import lr_scheduler
import time
import os
import copy
import torchvision


### 2) Hyperparameters

In [ ]:
batch_size = 2
learning_rate = 0.001
epoch = 10
num_category = 2

## 2. Data
### 1) Load images from folder

In [ ]:
#데이터 위치
data_dir = "./data"

data_transforms = { 
                #더 좋은 학습을 위해 데이터 증강기법 도입!
                'train': transforms.Compose([
                        transforms.RandomResizedCrop(224), #사이즈변환 224x224
                        transforms.RandomHorizontalFlip(), #임의의 횡축 접기
                        transforms.ToTensor(), #텐서로 바꾸기
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #정규화
                ]),
                'val': transforms.Compose([
                        transforms.Resize(256),#사이즈변환 256x256
                        transforms.CenterCrop(224), #중앙 잘라내기
                        transforms.ToTensor(), #텐서로 바꾸기
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
}

train_set = datasets.ImageFolder(data_dir, transform = data_transforms)
val_set = datasets.ImageFolder(data_dir, transform = data_transforms)
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir),
                                                data_transforms[x])
                                                for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("레이블 개수: ", dataset_sizes)
print("클래스 이름: ", class_names)
class_num = len(class_names)
print("클래스 개수: ", class_num)
print(device)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.axis('off')
    
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## 3. Model & Optimizer
### 1) ResNet

In [ ]:
# children() -> immediate children modules 
# modules() -> iterate all modules

model = models.resnet50(pretrained=True)

for name,module in model.named_children():
    print(name)

### 2) Finetuning the convnet

In [ ]:
# 가장 마지막 부분을 기존의 1000->2로 바꿔줍니다.
model = models.resnet18(pretrained=True)
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, class_num)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# 통계적 경사하강법실시합니다.
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#Adam도 좋은 학습 Optimizer입니다.
# optimizer_ft= optim.Adam(model.parameters(), lr=0.001)

# 학습률 감소를 매 7에폭마다 0.1씩 감소시켜줍니다.
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
#수업시간에 배운 위에는 전이학습을(Transfer Learning)을 이용하여 
#어느정도 학습 weight를 갱신시키고 싶지 않다면? 다음과 같은 주석을 해제시켜주면 됩니다.
# for param in model.parameters():
#     param.requires_grad = False

### 3) Model on GPU

In [ ]:
model.cuda()


In [ ]:
for name in model.children():
    print(name)

### 4) Loss & Optimizer

In [ ]:
# define loss func & optimizer
# model.parameters() also works because of the cell right above

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.layer1.parameters(), lr=learning_rate) 

In [ ]:
#학습 진행률 코드관련
import re
from tqdm import trange

class DescStr:
	def __init__(self):
	  self._desc = ''
	
	def write(self, instr):
	  self._desc += re.sub('\n|\x1b.*|\r', '', instr)
	
	def read(self):
	  ret = self._desc
	  self._desc = ''
	  return ret
	
	def flush(self):
	  pass

## 4. Train

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    tbar = trange(num_epochs)
    desc = DescStr()
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in tbar:
        #print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        #print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # 각 Epoch당 돌아줍니다.
            for inputs, labels in dataloaders[phase]:
                tbar.set_description(desc.read()) #진행률관련
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 기울기 감소법
                optimizer.zero_grad()

                # 포워드
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) #가장 높은 확률의 예측값 골라내기
                    loss = criterion(outputs, labels)
                    
                    # 역전파, 최적화 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # 정확도 관련
                running_loss += loss.item() * inputs.size(0)
                #맞는것들이 있는것들만 추려내기
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            
            #에폭 손실 체크
            epoch_loss = running_loss / dataset_sizes[phase]
            #정확도체크
            epoch_acc = running_corrects.double() / dataset_sizes[phase]  
            tbar.set_postfix(epoch = f'{epoch}/{num_epochs}', \
                            # 표시하고 싶은 변수 입력
                            loss = epoch_loss, accuracy = epoch_acc.item()* 100.) 
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # 최고의 모델을 반환시켜줍니다.
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=epoch)

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model)
plt.ioff()
plt.show()

## 6. Test

In [ ]:
data_dir = './data'
#테스트 데이터는 데이터 증강을 하지 않습니다.
test_transform = transforms.Compose([
                        transforms.Resize((224, 224)), #사이즈변환 255x255 
                        transforms.ToTensor(), #텐서로 바꾸기
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), #정규화
                ])

test_set = datasets.ImageFolder(data_dir, transform = test_transform)
test_batch = DataLoader(test_set, batch_size=batch_size,
                            shuffle=True, num_workers=2)

In [ ]:
#최종 테스트
model.eval()
correct = 0
total = 0

for img,label in test_batch:
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    
    output = model(img)
    _, pred = torch.max(output.data,1)
    
    total += label.size(0)
    correct += (pred == label.data).sum()   

print("Accuracy: {}".format(correct/total))

# 7. Model Save
### 모델을 저장해 봅시다.

In [ ]:
#모델 저장하기
torch.save(model, './model.pth')


# 8. Model Load
### 모델을 불러와 봅시다.

In [ ]:
#빈 모델 불러오기
model = models.resnet50(pretrained=False)
# 가장 마지막 부분을 기존의 1000->2로 바꿔줍니다.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, class_num)
model = model.to(device)
#모델안의 값을 출력
for idx, i in enumerate(model.parameters()):
    print(i)


In [ ]:
#그동안 학습한 값을 확인
model = (torch.load('./model.pth', map_location="cuda:0"))  # 사용할 GPU 장치 번호를 선택합니다.
for idx, i in enumerate(model.parameters()):
    print(i)


In [ ]:
        
device = torch.device("cuda")
model.to(device)

## 사진을 찍어 관찰해보세요!
### 사진을 직접 찍고 결과를 관찰해 봅시다.


In [ ]:
def predict(image, model):
    # Pass the image through our model
    output = model.forward(image)
    
    # Reverse the log function in our output
    output = torch.exp(output)
    
    # Get the top predicted class, and the output percentage for
    # 확률과 클래스를 반환
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

In [ ]:

def take_photo(filename=name+'_with_mask.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Take picture with mask';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
# 이미지 경로를 받아오면 딥러닝 형태로 변환해주는 함수
def process_image(image_path):
    # Load Image
    img = Image.open(image_path)
    
    # 사이즈
    width, height = img.size
    img = img.resize((255, int(255*(height/width))) if width < height else (int(255*(width/height)), 255)) # 255 x 255 사이즈로 변환
    
    # 변화된 사이즈
    width, height = img.size
    left = (width - 224)/2
    top = (height - 224)/2
    right = (width + 224)/2
    bottom = (height + 224)/2
    img = img.crop((left, top, right, bottom))
    
    img = np.array(img)
    print(img.shape)
    # Chnnel이 먼저되게끔 설정(tensor로 변환)
    img = img.transpose((2, 0, 1))
    
    # 0~1로 변환
    img = img/255
    
    # Normalize 실시
    img[0] = (img[0] - 0.485)/0.229
    img[1] = (img[1] - 0.456)/0.224
    img[2] = (img[2] - 0.406)/0.225
    
    # 배치사이즈삽입 1
    img = img[np.newaxis,:]
    
    # Turn into a torch tensor
    image = torch.from_numpy(img)
    image = image.float()
    return image

In [ ]:
# Process Image
image = process_image("data/dogs/retriever.jpg")
# Give image to model to predict output

top_prob, top_class = predict(image.cuda(), model)
print(top_prob, top_class)
# Show the image

# Print the results
print("The model is ", top_prob*100, "% certain that the image has a predicted class of ", top_class)

In [ ]:
plt.imshow(Image.open("data/dogs/retriever.jpg"))
plt.axis('off')

# Pytorch모델을 라즈베리파이에 넣기위하여 소형화 시키기 위한 작업
## Pytorch는 매우크기 때문에 이를 저전력 모델에 넣기 위하여 소형화 시키는 작업을 해야합니다.
이를 위하여 Tensorflow-Lite라는 모델로 변환시켜주려고 합니다.
Pytorchs는 Facebook에서 만든 Framework이기 때문에 공동 Deep Learning 규약인 ONNX(Open standard for machine learning)를 거쳐
Google사의 Tensorflow 형태로 변형시켜준뒤에 Tensorflow-Lite로 변형시켜주고자 합니다.

In [ ]:
!python ./torch2tflite/setup.py install

In [ ]:
!python ./torch2tflite/torch2tflite/converter.py\
    --torch-path ./model.pth\
    --tflite-path model.tflite\
    --target-shape 224 224 3\
    --seed 10